# SEIRX model runs for test technology and screening frequency investigation

In [ ]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

## Parameters

### Simulation parameters and measures in place

In [ ]:
with open('params/testing_and_vaccination_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/testing_and_vaccination_measures.json') as json_file:
    measures = json.load(json_file)


In [ ]:
# paths for data I/O
contact_network_src = '../data/contact_networks'
ensmbl_dst = '../data/simulation_results/ensembles/testing_and_vaccination'

### Screening parameters

In [ ]:
test_types = ['same_day_antigen', 'same_day_LAMP']

vaccination_scenarios = [(0.0, 0.5), (0.5, 0.0), (0.5, 0.9), (0.9, 0.9)]

# resident and employee streening intervals (in days)
e_screen_range = [2, 3, 7, None]
r_screen_range = [2, 3, 7, None]

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

# ensemble_size
N_runs = 5000

screening_params = [(i, j, k, l, m, n, N_runs)\
              for i in test_types \
              for j in index_cases \
              for k in e_screen_range \
              for l in r_screen_range \
              for m, n in vaccination_scenarios]

print('there are {} parameter combinations'.format(len(screening_params)))

In [ ]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''

    # extract the simulation parameters from the parameter list
    test_type, index_case, e_screen_interval, r_screen_interval, \
        e_vaccination_ratio, r_vaccination_ratio, N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params, 
                contact_network_src, ensmbl_dst, test_type, index_case,
                e_screen_interval=e_screen_interval,
                r_screen_interval=r_screen_interval,
                e_vaccination_ratio=e_vaccination_ratio,
                r_vaccination_ratio=r_vaccination_ratio,
               )

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case, 
            test_type=test_type, e_screen_interval=e_screen_interval,
            r_screen_interval=r_screen_interval,
            e_vaccination_ratio=e_vaccination_ratio,
            r_vaccination_ratio=r_vaccination_ratio)

    return row

In [ ]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 2 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case', 'resident_screen_interval',
              'employee_screen_interval',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_testing_and_vaccination_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)